## Hardware-oriented implementation of the "count the ones" test
2023-08-10 Naoki F., AIT

ライセンス条件は LICENSE.txt を参照してください
（See LICENSE.txt for license information)

真性乱数生成器（TRNG）に対するドライバクラスを定義します．

A driver class for a true random number generator (TRNG) is defined.

In [ ]:
from pynq import DefaultIP

class TRNGCore(DefaultIP):
    ADDR_GO         = 0x00
    ADDR_STOP       = 0x00
    ADDR_RUN        = 0x00
    ADDR_OVER       = 0x00
    MASK_RUN        = 0x1
    MASK_OVER       = 0x2
    ADDR_SEND_BYTES = 0x04
    ADDR_SENT_BYTES = 0x04
    ADDR_DMA_BYTES  = 0x08
    ADDR_SUM_DATA   = 0x08
    ADDR_PARAMETER  = 0x0c
    ADDR_STATS      = 0x0c
    ADDR_STAT_ADDR  = 0x10
    send_bytes = 0
    
    def __init__(self, description):
        super().__init__(description=description)
        self.stop()
        
    bindto = ['xilinx.com:user:TC_TERO_IP:1.0']
    
    def start (self, total, dma_size = 4096):
        self.write(self.ADDR_SEND_BYTES, total)
        self.write(self.ADDR_DMA_BYTES, dma_size)
        self.write(self.ADDR_GO, 1)
        self.send_bytes = total
    
    def wait (self):
        if self.send_bytes != 0:
            while True:
                if (self.read(self.ADDR_RUN) & self.MASK_RUN) == 0:
                    break
        if (self.read(self.ADDR_OVER) & self.MASK_OVER) != 0:
            raise OSError("TRNG Buffer Overflow")
        
    def stop (self):
        self.write(self.ADDR_STOP, 0)
    
    parameter = property()
    @parameter.setter
    def parameter(self, value):
        self.write(self.ADDR_PARAMETER, value)
    
    @property
    def sent_bytes(self):
        return self.read(self.ADDR_SENT_BYTES)
    
    @property
    def sum_data(self):
        return self.read(self.ADDR_SUM_DATA)
    
    def get_user_stats(self, addr):
        self.write(self.ADDR_STAT_ADDR, addr)
        return self.read(self.ADDR_STATS)


定数を宣言します．TC_TERO_PARAM は 0～1048575 の範囲で任意に選べます．

Constants are defined. You may select an arbitrary integer from 0 to 1048575 as TC-TERO_PARAM.

In [ ]:
NUM_BLOCKS = 41
NUM_WORDS = 1561
TOTAL_BITS = NUM_BLOCKS * NUM_WORDS * 32 # 41 x 1561 x 32 = 2,048,032 bit
TC_TERO_PARAM = 1

オーバーレイを読み出し，バッファなどの準備を行います．

The hardware overlay is opened and buffers are allocated.

In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np

pl = Overlay("TRNG_IP_MOD.bit")
rng = pl.TC_TERO_IP_0
tst = pl.countone_0
dma = pl.axi_dma_0

buffer_r = allocate(shape=(NUM_WORDS,), dtype=np.uint32)
buffer_w = allocate(shape=(NUM_WORDS,), dtype=np.uint32)

rng.parameter = TC_TERO_PARAM

乱数を生成し，その結果を random.dat に保存しながら，乱数検定を行います．

We now generate random numbers and save them as random.dat, while performing randomness test.

In [ ]:
file = open("random.dat", "wb")
tst.register_map.CTRL.AP_START = 1
rng.start(NUM_BLOCKS * NUM_WORDS * 4, NUM_WORDS * 4)

for i in range(NUM_BLOCKS):
    dma.recvchannel.transfer(buffer_r)
    if i != 0:
        buffer_w.tofile(file)
    dma.recvchannel.wait()
    buffer_r, buffer_w = buffer_w, buffer_r
    
rng.stop()
buffer_w.tofile(file)
file.close()
buffer_r.freebuffer()
buffer_w.freebuffer()

TERO 型真性乱数生成器では，発振の回数の偶奇を 0, 1 のランダムビットとして出力します．
（発振の回数が 255 以上の場合は 255 として扱います）
回路内で発振の回数の分布を統計情報として記録しているので，それを取り出し，プロットしてみます．

In TERO-based TRNG, a random output bit is defined as the number of oscillations modulo two.
(If the number of oscillations is greater than 255, it is recorded as 255.)
As the distribution of the number of oscillation is recorded by the TRNG circuit as statistics, we extract and plot it.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

stats = [0]
for i in range(1, 256):
    stats.append(rng.get_user_stats(i) / TOTAL_BITS)
plt.bar(list(range(256)), stats, width=1)
plt.show()

併せて，count the ones テストを実行した結果を取り出し，乱数検定の合否を判定します．
検定回路から出力された統計量は，もし真性乱数生成器から偏りのない乱数が生成されていれば，平均2500，標準偏差√5000の正規分布に近似的に従います．
これをもとに統計量からP値を求めます．

Also, we determine pass or fail of the count-the-ones test.
The statistic from the test circuit will, if the TRNG is free from bias, follow a normal distribution with a mean of 2500 and standard deviation of sqrt(5000).
We calculate the p-value from the statistic.

In [ ]:
from scipy.stats import norm

stat = int(tst.register_map.ap_return) / 65536
print(f"統計量 (statistic): {stat}")

p = norm.pdf((2500 - stat) / np.sqrt(5000))
print(f"P値 (p-value): {p}")